In [1]:
#re.findall('\(.*?\)',b1)
#re.findall('\(.*?\)',f1)

In [300]:
print('ha')

ha


In [199]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import copy

from goodreads import client
gc = client.GoodreadsClient('9addHIFaPJmit7dzC5ZA', 'iDXvx13Rnt1iJkWOxRkSh9wLnbFWmMNgCUMFLJmLPo')

from imdb import IMDb

In [3]:
wikipages = ['https://en.wikipedia.org/wiki/List_of_children%27s_books_made_into_feature_films?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(0%E2%80%939,_A%E2%80%93C)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(D%E2%80%93J)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(K%E2%80%93R)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(S%E2%80%93Z)?oldformat=true'
            ]

In [4]:
def add_to_rows(page_soup):
    """
    Method extracts row elements from the soup tags of individual 
    list pages (A - F, H - J, etc.) and appends them to the `rows` 
    list.
    
    Args:
        page_soup: (BeautifulSoup tag) soup for list page.
    """
    
    global rows
    
    tables = page_soup.find_all('table', {'class': 'wikitable'})

    for alphabet in tables:
        rows += alphabet.findAll('tr')
    
    print("Tables added: {}, Rows added: {}".format(len(tables), len(rows)))

In [166]:
def working_wiki_link(wiki_link):
    """
    Converts relative URLs to global URLs.
    """
    if '/wiki/' in wiki_link:
        return 'https://en.wikipedia.org' + wiki_link
    return wiki_link

In [477]:
def get_adaptations(book_title, href_list):
    """
    Parses through list of movies associated with a specific `book_title`,
    adds movies to `movie_book_dict`, and returns list of movies.
    Adds year to the movie title if information available. 
    
    Args:
        book_title: (str) name of original book.
        href_list: (list) Result of findAll('a') on the movies column.
        
    Returns:
        adaptations: List of movie title names.
    """
    adaptations = []
    
    if len(href_list) == 1:
        adaptations.append(href_list[0]['title'])
        return adaptations
        
    for index, tag in enumerate(href_list):
        year_search = re.search(r'\d\d\d\d', tag['href'])
        
        if year_search is not None:  #There is a 4 digit number in the title
            year_search = year_search.group()
        
            if year_search not in tag['title']:
                version = tag['title'] + ' (' + year_search + ')'
            else:
                version = tag['title']
            
            adaptations.append(version)
            
        
        else:
            version = tag['title']
            adaptations.append(version)
        
        movie_book_dict[working_wiki_link(tag['href'])] = {'movie_title': version, 
                                                           'book_title': book_title,
                                                           'imdb_no': '', 'usable': '',
                                                           'isbn': '', 'oclc': ''}

    
    return adaptations

def clean_adaptations(adaptations):
    """
    Makes sure the list of adaptations doesn't contain TV series or miniseries.
    """
    
    cleaned = []
    
    for title in adaptations:
        if 'TV' in title or 'miniseries' in title:
            continue
        if 'page does not exist' in title:
            continue
        else:
            cleaned.append(title)
            
    return cleaned

In [296]:
def add_to_book_movie_dict():
    """
    Method parses newly updated rows for book/film title, book author,
    number of adaptions, and list of adaptations, and adds them to the
    `book_movie_dict` as a nested dictionary. 
    global sum_adaptations
    """
    global sum_adaptations
    
    for row in rows:
        cols = row.findAll('td')

        try:
            if (len(cols[0].findAll('a')) == 2) and (cols[1].find('a') is not None):
                series = cols[0].text.replace('\n', '').split(',')[0]

                book_title = cols[0].findAll('a')[0]['title']
                author = cols[0].findAll('a')[-1]['title']

                book_wiki_url = working_wiki_link(cols[0].findAll('a')[0]['href'])

                try:
                    adaptations = clean_adaptations(get_adaptations(book_title, cols[1].findAll('a')))
                except KeyError:
                    adaptations = []
                    pass

                sum_adaptations += len(adaptations)

                book_movie_dict_all[series] = {'author': author,
                                            'book_title': book_title,
                                            'book_wiki_url': book_wiki_url,
                                            'count': len(adaptations),
                                            'adaptations': adaptations,
                                            'isbn': '', 'oclc':'', 'valid_identifier': ''}

        except IndexError or KeyError:
            pass
        
    print("No. of rows/original books in dict: {}, No. of adaptations: {} \n"
          .format(len(book_movie_dict_all), sum_adaptations))

In [153]:
rows = []
book_movie_dict_all = {}
movie_book_dict = {}
sum_adaptations = 0

for url in wikipages:
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'lxml')
    
    add_to_rows(soup)
    add_to_book_movie_dict()

Tables added: 25, Rows added: 376
No. of rows/original books in dict: 259, No. of adaptations: 754 

Tables added: 3, Rows added: 798
No. of rows/original books in dict: 553, No. of adaptations: 2046 

Tables added: 7, Rows added: 1245
No. of rows/original books in dict: 813, No. of adaptations: 3917 

Tables added: 8, Rows added: 1662
No. of rows/original books in dict: 1056, No. of adaptations: 6426 

Tables added: 7, Rows added: 2034
No. of rows/original books in dict: 1237, No. of adaptations: 9466 



In [28]:
len_initial = len(book_movie_dict_all)
len_initial

1237

In [30]:
movie_book_dict['https://en.wikipedia.org/wiki/102_Dalmatians'], movie_book_dict['https://en.wikipedia.org/wiki/3_Godfathers']

({'movie_title': '102 Dalmatians', 'book': 'The Hundred and One Dalmatians'},
 {'movie_title': '3 Godfathers', 'book': 'Three Godfathers'})

## Don't touch above

In [428]:
def get_isbn_oclc(wiki_url):
    """
    Parses the `infobox vcard` element of the wikipedia page of a book, 
    and returns the 10-digit or 13-digit ISBN number (if found). 
    
    Args:
        wiki_url (str): Valid (not local) wikipedia link.

    Returns:
        isbn: Returns ISBN code is parsed correctly, 'broken' if formmated 
        incorrectly or otherwise. The code is de-hyphentated for later pipeline
        efficiency. 
 
    """
    try:
        page = requests.get(wiki_url).text
        soup = BeautifulSoup(page, 'lxml')
    except MissingSchema:
        return 'broken'  
    
    infobox = soup.find('table', {'class': 'infobox vcard'})
    isbn, oclc = '', ''
    
    if infobox is None:
        isbn, oclc = 'broken', 'broken'
    
    else:
        isbn_found, oclc_found = False, False
    
        for row in infobox.findAll('tr'):
        #ISBN directly available
            if 'ISBN' in row.text and isbn_found is False:
                isbn = re.sub('[^0-9]','', row.findAll('a')[-1].text)
                isbn_found = True
        
            if row.find('a', {'title': 'OCLC'}) is not None and oclc_found is False:
                oclc = row.find('td').text
                oclc = oclc.split()[0]
                oclc_found = True
    
        if isbn_found is False and oclc_found is False:
            isbn, oclc = 'broken', 'broken' 
        
    return {'isbn': isbn, 'oclc': oclc}    

In [ ]:
broken_count = 0
usable_count = 0
index = 0
book_movie_dict = {}

for key in book_movie_dict_all.keys():
    
    link = book_movie_dict_all[key]['book_wiki_url']
    
    try:
        isbn_oclc = get_isbn_oclc(link)
        
        book_movie_dict_all[key]['isbn'] = isbn_oclc['isbn']
        book_movie_dict_all[key]['oclc'] = isbn_oclc['oclc']
    
        if book_movie_dict_all[key]['isbn'] == 'broken' and book_movie_dict_all[key]['oclc'] == 'broken':
            book_movie_dict_all[key]['valid_identifier'] = False
                #Nothing changes, the book is unusable, and the 'usable' flag remains False
        else:
            book_movie_dict_all[key]['valid_identifier'] = True
            
            
            
            book_movie_dict[key] = copy.deepcopy(book_movie_dict_all[key]) #Deepcopy 'true' values to another dictionary
            usable_count += 1
    except NameError:
        pass
        
    if (index % 100 == 0):
        broken_count = index - usable_count
        print("Usable books: {}, Broken books: {}, Total books: {}".format(usable_count, broken_count, index))
    
    index += 1
    
print("Usable books: {}, Broken books: {}, Total books: {}".format(usable_count, broken_count, len_initial))

Usable books: 0, Broken books: 0, Total books: 0
Usable books: 50, Broken books: 50, Total books: 100


In [155]:
len(book_movie_dict), len(movie_book_dict)

(501, 1435)

In [157]:
safe_movie_book_dict = copy.deepcopy(movie_book_dict)
id(safe_movie_book_dict), id(movie_book_dict)

(4900115728, 4847924352)

we lose a bunch of old books, but it's okay since inflation would have destroyed old movie sales as well. i guess it wokrs out

## Don't touch above

In [62]:
book_movie_dict['Dreamcatcher (2001)']

{'author': 'Stephen King',
 'book_title': 'Dreamcatcher (novel)',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/Dreamcatcher_(novel)',
 'count': 1,
 'adaptations': ['Dreamcatcher (2003 film)'],
 'isbn': '9780743211383',
 'oclc': '',
 'valid_identifier': '',
 'valid_identifer': True}

In [63]:
book_movie_dict["Gulliver's Travels (1726)"]

KeyError: "Gulliver's Travels (1726)"

In [64]:
book_movie_dict_all["Gulliver\'s Travels (1726)"]

{'author': 'Jonathan Swift',
 'book_title': "Gulliver's Travels",
 'book_wiki_url': 'https://en.wikipedia.org/wiki/Gulliver%27s_Travels',
 'count': 7,
 'adaptations': ['Le Voyage de Gulliver à Lilliput et chez les géants (1902 film)',
  'The New Gulliver',
  "Gulliver's Travels (1939 film)",
  'The Three Worlds of Gulliver',
  "Gulliver's Travels Beyond the Moon",
  "Gulliver's Travels (1977 film)",
  "Gulliver's Travels (2010 film)"],
 'isbn': 'broken',
 'oclc': 'broken',
 'valid_identifier': '',
 'valid_identifer': False}

In [65]:
desired_metadata = ['average_rating', 
                    'publication_date',
                    'rating_dist',
                    'ratings_count', 'format',
                    'text_reviews_count',
                    'title', 'language_code']

def get_goodreads_data(gc_code):
    """
    Add additional review and ratings related meta_data from the GoodReads API.
    
    Args:
        gc_code: Strictly Goodreads code. Can be 10-digit or 13-digit.
        
    Returns:
        gr_metadata: Dict with additional metadata.
    """
    
    gr_metadata = {}
    try:
        book = gc.book(gc_code)
    except NameError:
        return gr_metadata
    except ExpatError:
        return gr_metadata
    
    for attrs in book.__dict__['_book_dict']:
        if attrs in desired_metadata:
            gr_metadata['gc_' + attrs] = book.__dict__['_book_dict'][attrs]
            
    return gr_metadata

In [478]:
book_metadata_wanted = ['identifiers', 'number_of_pages', 'subject_places', 
                        'subjects', 'publish_date', 'publish_places']

def update_book_metadata(book_title):
    """
    Uses the OpenLibrary and Goodreads APIs (when possible) to 
    add metadata about the books to the `book_movie_dict`.
    
    Args: 
        book_title: (str) used when merging back into `book_movie_dict`.
    
    Returns:
        updated: (bool) True if metadata updated, False otherwise.
    
    
    """
    metadata = {}
    all_metadata = {}
    
    
    if book_movie_dict[book_title]['valid_identifier'] is False:
        #API can't be accessed
        metadata['metadata_updated'] = False
        book_movie_dict[book_title].update(metadata)
        return
    
    elif book_movie_dict[book_title]['isbn'] != '':
        code = book_movie_dict[book_title]['isbn']
        curl = 'https://openlibrary.org/api/books?bibkeys=ISBN:{}&jscmd=data&format=json'.format(code)
        all_metadata = requests.get(curl).json()
    else:
        code = book_movie_dict[book_title]['oclc']
        curl = 'https://openlibrary.org/api/books?bibkeys=OCLC:{}&jscmd=data&format=json'.format(code)
        all_metadata = requests.get(curl).json()
        
    if len(all_metadata) > 0:
        key = list(all_metadata.keys())[0]
        nested_keys = list(all_metadata[key].keys())
        
        for data_point in book_metadata_wanted:
            if data_point in nested_keys:
                metadata[data_point] = all_metadata[key][data_point]
    
        if 'subject_places' in nested_keys:
            metadata['subject_places'] = [place['name'][:30] for place in metadata['subject_places']]
        if 'subjects' in nested_keys:
            metadata['subjects'] = [subject['name'][:30] for subject in metadata['subjects']]
        if 'publish_places' in nested_keys:
            metadata['publish_places'] = [place['name'][:30] for place in metadata['publish_places']]            
        
        if 'identifiers' in nested_keys:
            if 'goodreads' in list(metadata['identifiers'].keys()):
                metadata['goodreads'] = metadata['identifiers']['goodreads'][0]
            metadata.pop('identifiers')
            
        if 'goodreads' in metadata.keys():
            metadata.update(get_goodreads_data(metadata['goodreads']))
            
        metadata['metadata_updated'] = True
        book_movie_dict[book_title].update(metadata)
        return
    
    else:
        metadata['metadata_updated'] = False
        book_movie_dict[book_title].update(metadata)
        return


In [67]:
update_book_metadata('Dreamcatcher (2001)')
book_movie_dict['Dreamcatcher (2001)']

{'author': 'Stephen King',
 'book_title': 'Dreamcatcher (novel)',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/Dreamcatcher_(novel)',
 'count': 1,
 'adaptations': ['Dreamcatcher (2003 film)'],
 'isbn': '9780743211383',
 'oclc': '',
 'valid_identifier': '',
 'valid_identifer': True,
 'number_of_pages': 620,
 'subject_places': ['Maine', 'Derry', 'Hole-in-the-Wall', 'Jefferson Tract'],
 'subjects': ['Internet Archive Wishlist',
  'male friendship',
  'life on other planets',
  'telepathy',
  'Down Syndrome',
  'psychological fiction',
  'extraterrestrial beings',
  'hunting',
  'friendship',
  'In library',
  'Accessible book',
  'Fiction',
  'Human-alien encounters',
  'Protected DAISY',
  'Fear',
  'Fantasy Fiction',
  'Roman fantastique',
  'Audiobooks',
  'hunting stories',
  'camping',
  'Male friendship'],
 'publish_date': '2001',
 'publish_places': ['New York, USA'],
 'goodreads': '441240',
 'gc_title': 'Dreamcatcher',
 'gc_language_code': 'eng',
 'gc_average_rating': '3.62',
 

don't run below till the end

In [68]:
from xml.parsers.expat import ExpatError

metadata_update_count = 0

for index, key in enumerate(book_film_dict.keys()):
    
    if book_film_dict[key]['metadata_updated'] is True:
        metadata_update_count += 1
    
    if index % 75 == 0 and index != 0:
        print("Indexed: {}, Updated: {}, Usable: {}%"
              .format(index, metadata_update_count, round(100*metadata_update_count/index, 3)))

Indexed: 75, Updated: 59, Usable: 78.667%
Indexed: 150, Updated: 125, Usable: 83.333%
Indexed: 225, Updated: 186, Usable: 82.667%
Indexed: 300, Updated: 250, Usable: 83.333%
Indexed: 375, Updated: 319, Usable: 85.067%
Indexed: 450, Updated: 383, Usable: 85.111%


In [69]:
print("Done. Started with {} rows; now {} are usable: ~{}%."
    .format(len_initial, metadata_update_count, round(100*metadata_update_count/len_initial, 3)))

Done. Started with 1237 rows; now 429 are usable: ~34.681%.


In [72]:
book_movie_dict['Grand Canary (1933)']

{'author': 'A. J. Cronin',
 'book_title': 'Grand Canary (novel)',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/Grand_Canary_(novel)',
 'count': 1,
 'adaptations': ['Grand Canary (film)'],
 'isbn': '0450020479',
 'oclc': '',
 'valid_identifier': '',
 'valid_identifer': True,
 'number_of_pages': 223,
 'subject_places': ['Canary Islands',
  'Gran Canaria',
  'Gran Canaria (Canary Islands)'],
 'subjects': ['Fiction', 'Physicians', 'Fiction in English'],
 'publish_date': '1975',
 'publish_places': ['London'],
 'goodreads': '184807',
 'metadata_updated': True}

## Moving on to movies

In [438]:
imdb_lists = [#'https://www.imdb.com/list/ls073821013/', 'https://www.imdb.com/list/ls050071819/',
             'https://www.imdb.com/list/ls068298300/']

In [472]:
def get_book_essentials_from_movie_title(movie_title):
    
    book_essentials = {}
    possible_wiki_combos = []
    isbn, oclc = '', ''
    valid_identifier = False
    
    # Assume that movie title is the book title
    book_title = movie_title
    wiki_link = 'https://en.wikipedia.org/wiki/' + book_title
    
    isbn_oclc = get_isbn_oclc(wiki_link)
    isbn = isbn_oclc['isbn']
    oclc = isbn_oclc['oclc']
               
    if isbn == 'broken' and oclc == 'broken':
        possible_wiki_combos.append(movie_title.replace('&', 'and'))
        possible_wiki_combos.append(movie_title + '_(novel)')
        possible_wiki_combos.append(movie_title + '_(book)')
 
        try:
            possible_wiki_combos.append(movie_title.split(':')[1].strip()) #Removes :
            possible_wiki_combos.append(movie_title.split(':')[1].split('-')[0].strip()) #Removes :
        except IndexError:
            pass
        
        for possible_title in possible_wiki_combos:
            wiki_link = 'https://en.wikipedia.org/wiki/' + possible_title
            isbn_oclc = get_isbn_oclc(wiki_link)
            isbn = isbn_oclc['isbn']
            oclc = isbn_oclc['oclc']
        
            if isbn != 'broken' or oclc != 'broken':
                book_title = possible_title
                valid_identifier = True
                break
            else:
                wiki_link = 'https://en.wikipedia.org/wiki/The ' + possible_title
                isbn_oclc = get_isbn_oclc(wiki_link)
                isbn = isbn_oclc['isbn']
                oclc = isbn_oclc['oclc']
        
                if isbn != 'broken' or oclc != 'broken':
                    book_title = possible_title
                    valid_identifier = True
                    break
    else:
        valid_identifier = True
                
    if valid_identifier is False:
        wiki_link, book_title = '', 'unknown'

    book_essentials = {'author': '',
                       'book_title': book_title,
                       'book_wiki_url': wiki_link,
                       'count': 1,
                       'adaptations': [movie_title],
                       'isbn': isbn,
                       'oclc': oclc,
                       'valid_identifier': valid_identifier} 
    
    return (book_essentials)
    
get_book_essentials_from_movie_title("Percy Jackson: Sea of Monsters")

{'author': '',
 'book_title': 'Sea of Monsters',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/The Sea of Monsters',
 'count': 1,
 'adaptations': ['Percy Jackson: Sea of Monsters'],
 'isbn': '0786856866',
 'oclc': '64664383',
 'valid_identifier': True}

In [489]:
def add_imdb_lists(imdb_list):
    new_books = {}
    new_movies = {}
    
    page = requests.get(imdb_list).text
    soup = BeautifulSoup(page, 'lxml')
    
    list_elements = soup.findAll('h3', {'class': 'lister-item-header'})
    list_elements = [l.find('a') for l in list_elements]
    
    for l in list_elements:
        imdb_no = l['href'].split('/')[2]
        imdb_link = 'https://imdb.com' + l['href']
        
        movie_title = l.text
        
        #Get details about new books
        book_essentials = get_book_essentials_from_movie_title(movie_title)
        new_books[book_essentials['book_title']] = book_essentials
        
        movie_essentials = {'movie_title': movie_title, 
                            'book_title': book_essentials['book_title'],
                            'imdb_no': imdb_no,
                            'isbn': book_essentials['isbn'],
                            'oclc': book_essentials['oclc'],
                            'usable': True}
        
        new_movies[imdb_link] = movie_essentials
    #add new books to book_film dict
    #add new movies to movie_film dict
    
    return new_movies
    #return movie_list

In [491]:
test  = 'https://www.imdb.com/list/ls048749162/'
x = add_imdb_lists(test)
x

{'https://imdb.com/title/tt0814255/': {'movie_title': 'Percy Jackson & the Olympians: The Lightning Thief',
  'book_title': 'The Lightning Thief',
  'imdb_no': 'tt0814255',
  'isbn': '0786856297',
  'oclc': '60786141',
  'usable': True},
 'https://imdb.com/title/tt1790864/': {'movie_title': 'The Maze Runner',
  'book_title': 'The Maze Runner',
  'imdb_no': 'tt1790864',
  'isbn': '9780385737944',
  'oclc': '299381315',
  'usable': True},
 'https://imdb.com/title/tt0120338/': {'movie_title': 'Titanic',
  'book_title': 'unknown',
  'imdb_no': 'tt0120338',
  'isbn': 'broken',
  'oclc': 'broken',
  'usable': True},
 'https://imdb.com/title/tt0454876/': {'movie_title': 'Life of Pi',
  'book_title': 'Life of Pi',
  'imdb_no': 'tt0454876',
  'isbn': '0676973760',
  'oclc': '46624335',
  'usable': True},
 'https://imdb.com/title/tt8772262/': {'movie_title': 'Midsommar',
  'book_title': 'unknown',
  'imdb_no': 'tt8772262',
  'isbn': 'broken',
  'oclc': 'broken',
  'usable': True},
 'https://imdb

In [134]:
def get_imdb_no(wiki_movie):
    """
    Returns movie's respective IMDB no (if found). 
    
    Args: 
        wiki_movie: (str) Wikipedia link of movie's article.
        
    Returns:
        imdb_no: (str) IMDB no. if found, 'broken' otherwise.
    """
    imdb_no = ''   
   
    try:
        page = requests.get(wiki_movie).text
        soup = BeautifulSoup(page, 'lxml')
    except errors:
        return 'broken'
    
    external_links = soup.find('a', href=re.compile("imdb.com/title"))
    
    try:
        imdb_no = external_links['href'].split('/')[4]
    except TypeError:
        return 'broken'
    
    return imdb_no
    
get_imdb_no('https://en.wikipedia.org/wiki/Charlotte%27s_Web_(2006_film)')
    

'tt0413895'

In [256]:
from socket import gaierror

usable_count = 0
errors = (requests.exceptions.MissingSchema or 
          socket.gaierror or 
          KeyError or 
          requests.exceptions.ConnectionError or 
          requests.exceptions.NewConnectionError or 
          requests.exceptions.MaxRetryError)

for index, key in enumerate(movie_book_dict.keys()):
    movie_book_dict[key]['usable'] = False #default is false
    
    try:
        movie_book_dict[key]['imdb_no'] = get_imdb_no(key)
    except errors: 
        movie_book_dict[key]['imdb_no'] = 'broken'
    
    if movie_book_dict[key]['imdb_no'] != 'broken':
        movie_book_dict[key]['usable'] = True
        usable_count += 1
           
    if (index + 1) % 150 == 0:
        print('Index: {}, Usable: {} --> {}%'
              .format(index + 1, usable_count, round(usable_count*100/(index+1), 3)))    

Index: 150, Usable: 142 --> 94.667%
Index: 300, Usable: 283 --> 94.333%
Index: 450, Usable: 427 --> 94.889%
Index: 600, Usable: 567 --> 94.5%
Index: 750, Usable: 712 --> 94.933%
Index: 900, Usable: 855 --> 95.0%
Index: 1050, Usable: 995 --> 94.762%
Index: 1200, Usable: 1139 --> 94.917%
Index: 1350, Usable: 1269 --> 94.0%


In [257]:
movie_book_dict['https://en.wikipedia.org/wiki/1984_(1956_film)']

{'movie_title': '1984 (1956 film)',
 'book': 'Nineteen Eighty-Four',
 'imdb_no': 'tt0048918',
 'usable': True}

In [258]:
#DON'T RUN AGAIN
safe_movie_book_dict = copy.deepcopy(movie_book_dict)
id(safe_movie_book_dict), id(movie_book_dict)

(5045470408, 5007270824)

In [268]:
#fail safe
movie_book_dict = copy.deepcopy(safe_movie_book_dict)
id(safe_movie_book_dict), id(movie_book_dict)

(5045470408, 4875746040)

In [523]:
movie_metadata_wanted = ['Title', 'Year', 'Rated', 'Released', 'Runtime', 
                         'English', 'imdbRating', 'imdbVotes']

def add_imdb_metadata(movie_wiki_url):
    """
    
    """
    metadata = {}
    all_metadata = {}
    imdb_no = movie_book_dict[movie_wiki_url]['imdb_no']
    
    all_metadata = requests.get('http://www.omdbapi.com/?i={}&apikey=2cb213a7'.format(imdb_no)).json()
    
    for key in all_metadata.keys():
        if key in movie_metadata_wanted:
            metadata['imdb_' + key.lower()] = all_metadata[key]
            
        if 'Actors' in all_metadata.keys():
            metadata['imdb_actors'] = get_imdb_actors(all_metadata['Actors'])
        if 'Writer' in all_metadata.keys():
            metadata['imdb_writers'] = get_imdb_writers(all_metadata['Writer'])
        if 'Genre' in all_metadata.keys():
            metadata['imdb_genres'] = get_imdb_genres(all_metadata['Genre'])
            
    metadata.update(get_box_office(imdb_no))
            
    metadata['metadata_updated'] = True
    movie_book_dict[movie_wiki_url].update(metadata)

In [527]:
def get_box_office(imdb_no):
    
    imdb_page = 'https://www.imdb.com/title/' + imdb_no
    page = requests.get(imdb_page).text
    soup = BeautifulSoup(page, 'lxml')
    
    wanted = ['Budget', 'Opening Weekend USA', 'Gross USA', 'Cumulative Worldwide Gross']
    box_office_stats = {}
    
    for div in soup.findAll('div', {'class': 'txt-block'}):
        try:
            subheading = div.find('h4').text.split(':')[0]
        except AttributeError:
            subheading = None
            pass
        
        if subheading in wanted:
            money = div.text.split(':')[1].split('\n')[0]
            money = money.replace(',', '').split()[0].split('$')[1]
            
            subheading = subheading.lower().replace(' ', '_')
            box_office_stats[subheading] = money
    
    return box_office_stats

get_box_office('tt0454876')
    

{'budget': '120000000',
 'opening_weekend_usa': '22451514',
 'gross_usa': '124987023',
 'cumulative_worldwide_gross': '609016565'}

In [270]:
def get_imdb_actors(actor_string):
    actors = actor_string.rstrip('.').split(',')
    actors = [a.lower().strip() for a in actors]    
    return actors

def get_imdb_writers(writer_string):
    writers = writer_string.rstrip('.').split(',')
    writers = [re.sub(r'\([^()]*\)', '', w) for w in writers]
    writers = [w.lower().strip() for w in writers] 
    return writers

def get_imdb_genres(genre_string):
    genres = genre_string.rstrip('.').split(',')
    genres = [g.lower().strip() for g in genres]    
    return genres

In [271]:
movie_book_dict['https://en.wikipedia.org/wiki/Jaws_(film)']

{'movie_title': 'Jaws (film)',
 'book': 'Jaws (novel)',
 'imdb_no': 'tt0073195',
 'usable': True}

In [528]:
add_imdb_metadata('https://en.wikipedia.org/wiki/Jaws_(film)')
movie_book_dict['https://en.wikipedia.org/wiki/Jaws_(film)']

{'movie_title': 'Jaws (film)',
 'book': 'Jaws (novel)',
 'imdb_no': 'tt0073195',
 'usable': True,
 'imdb_title': 'Jaws',
 'imdb_actors': ['roy scheider',
  'robert shaw',
  'richard dreyfuss',
  'lorraine gary'],
 'imdb_writers': ['peter benchley', 'carl gottlieb', 'peter benchley'],
 'imdb_genres': ['adventure', 'drama', 'thriller'],
 'imdb_year': '1975',
 'imdb_rated': 'PG',
 'imdb_released': '20 Jun 1975',
 'imdb_runtime': '124 min',
 'imdb_imdbrating': '8.0',
 'imdb_imdbvotes': '511,159',
 'metadata_updated': True,
 'imdb_boxoffice': 'N/A',
 'Budget': '8000000',
 'Opening Weekend USA': '7061513',
 'Gross USA': '260000000',
 'Cumulative Worldwide Gross': '470653000',
 'budget': '8000000',
 'opening_weekend_usa': '7061513',
 'gross_usa': '260000000',
 'cumulative_worldwide_gross': '470653000'}

In [273]:
updated_count = 0
for index, key in enumerate(movie_book_dict.keys()):
    
    if movie_book_dict[key]['usable'] is True:
        try:
            add_imdb_metadata(key)
        except json.JSONDecodeError:
            movie_book_dict[key]['usable'] = False
            movie_book_dict[key]['imdb_no'] = 'broken'
            updated_count -= 1
            pass
        
        updated_count += 1
        
    if index % 150 == 0 and index != 0:
        print('Index: {}, Updated: {} --> {}%'.format(index, updated_count, round(updated_count * 100/index), 3))
        
print('Index: {}, Updated: {} --> {}%'.format(index, updated_count, round(updated_count * 100/index), 3))

Index: 150, Updated: 143 --> 95%
Index: 300, Updated: 284 --> 95%
Index: 450, Updated: 428 --> 95%
Index: 600, Updated: 568 --> 95%
Index: 750, Updated: 713 --> 95%
Index: 900, Updated: 856 --> 95%
Index: 1050, Updated: 996 --> 95%
Index: 1200, Updated: 1139 --> 95%
Index: 1350, Updated: 1269 --> 94%
Index: 1434, Updated: 1351 --> 94%


In [274]:
movie_book_dict['https://en.wikipedia.org/wiki/It_(2017_film)']

{'movie_title': 'It (2017 film)',
 'book': 'It (novel)',
 'imdb_no': 'tt1396484?d=inprod',
 'usable': True,
 'metadata_updated': True}

In [499]:
movie_book_dict['https://en.wikipedia.org/wiki/102_Dalmatians']

{'movie_title': '102 Dalmatians',
 'book': 'The Hundred and One Dalmatians',
 'imdb_no': 'tt0211181',
 'usable': True,
 'imdb_title': '102 Dalmatians',
 'imdb_actors': ['glenn close',
  'gérard depardieu',
  'ioan gruffudd',
  'alice evans'],
 'imdb_writers': ['dodie smith',
  'kristen buckley',
  'brian regan',
  'kristen buckley',
  'brian regan',
  'bob tzudiker',
  'noni white'],
 'imdb_genres': ['adventure', 'comedy', 'family'],
 'imdb_year': '2000',
 'imdb_rated': 'G',
 'imdb_released': '22 Nov 2000',
 'imdb_runtime': '100 min',
 'imdb_imdbrating': '4.9',
 'imdb_imdbvotes': '31,929',
 'metadata_updated': True}

In [301]:
len(movie_book_dict), len(book_movie_dict)

(1435, 500)

## Bringing in to pandas

In [306]:
book_film_cols = set()
movie_book_cols = set()

for index, key in enumerate(book_movie_dict.keys()):
    for nested_key in book_movie_dict[key].keys():
        book_film_cols.add(nested_key)
    
    if (index > 50):
        break
        
for index, key in enumerate(movie_book_dict.keys()):
    for nested_key in movie_book_dict[key].keys():
        movie_book_cols.add(nested_key)
    
    if (index > 50):
        break
        
book_film_cols, movie_book_cols

({'adaptations',
  'author',
  'book_title',
  'book_wiki_url',
  'count',
  'isbn',
  'oclc',
  'valid_identifer',
  'valid_identifier'},
 {'book',
  'imdb_actors',
  'imdb_genres',
  'imdb_imdbrating',
  'imdb_imdbvotes',
  'imdb_no',
  'imdb_rated',
  'imdb_released',
  'imdb_runtime',
  'imdb_title',
  'imdb_writers',
  'imdb_year',
  'metadata_updated',
  'movie_title',
  'usable'})

In [316]:
book_movie = pd.DataFrame.from_dict(book_movie_dict, orient='index')
book_movie.head(5)

,author,book_title,book_wiki_url,count,adaptations,isbn,oclc,valid_identifier,valid_identifer
'Salem's Lot (1975),Stephen King,'Salem's Lot,https://en.wikipedia.org/wiki/%27Salem%27s_Lot,1,[A Return to Salem's Lot],9780385007511,,True,True
A Clockwork Orange (1962),Anthony Burgess,A Clockwork Orange (novel),https://en.wikipedia.org/wiki/A_Clockwork_Oran...,2,"[Vinyl (1965 film), A Clockwork Orange (film)]",0434098000,,True,True
A Day with Wilbur Robinson (1990),William Joyce (writer),A Day with Wilbur Robinson,https://en.wikipedia.org/wiki/A_Day_with_Wilbu...,1,[Meet the Robinsons],0064433390,,True,True
A Judgement in Stone (1977),Ruth Rendell,A Judgement in Stone,https://en.wikipedia.org/wiki/A_Judgement_in_S...,1,[La Cérémonie],0091290708,,True,True
A Monster Calls (2011),Patrick Ness,A Monster Calls,https://en.wikipedia.org/wiki/A_Monster_Calls,1,[A Monster Calls (film)],9781406311525,,True,True


In [531]:
movie_book = pd.DataFrame.from_dict(movie_book_dict, orient='index')
movie_book[744:750]

,movie_title,book,imdb_no,usable,imdb_title,imdb_actors,imdb_writers,imdb_genres,imdb_year,imdb_rated,...,metadata_updated,imdb_boxoffice,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,budget,opening_weekend_usa,gross_usa,cumulative_worldwide_gross
https://en.wikipedia.org/wiki/Psycho_IV:_The_Beginning,Psycho IV: The Beginning,Psycho (novel),tt0102724,True,Psycho IV: The Beginning,"[anthony perkins, henry thomas, olivia hussey,...","[joseph stefano, robert bloch]","[horror, mystery, thriller]",1990,R,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://en.wikipedia.org/wiki/Puss_in_Boots_(2011_film),Puss in Boots (2011 film),Shrek!,tt0448694,True,Puss in Boots,"[antonio banderas, salma hayek, zach galifiana...","[tom wheeler, brian lynch, william davies, tom...","[animation, action, adventure, comedy, family,...",2011,PG,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://en.wikipedia.org/wiki/Quasimodo_d%27El_Paris,Quasimodo d'El Paris,The Hunchback of Notre-Dame,tt0163166,True,Quasimodo d'El Paris,"[patrick timsit, richard berry, mélanie thierr...","[jean-françois halin, victor hugo, raffy shart...","[comedy, crime]",1999,N/A,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://en.wikipedia.org/wiki/Que_la_b%C3%AAte_meure,Que la bête meure,The Beast Must Die (novel) (page does not exist),tt0064861,True,This Man Must Die,"[michel duchaussoy, caroline cellier, jean yan...","[nicholas blake, claude chabrol, paul gégauff,...","[drama, thriller]",1969,GP,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://en.wikipedia.org/wiki/Race_to_Witch_Mountain,Race to Witch Mountain,Escape to Witch Mountain,tt1075417,True,Race to Witch Mountain,"[dwayne johnson, annasophia robb, alexander lu...","[matt lopez, mark bomback, matt lopez, alexand...","[action, adventure, family, fantasy, sci-fi, t...",2009,PG,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://en.wikipedia.org/wiki/Raja_Aur_Runk,Raja Aur Runk,The Prince and the Pauper,tt0214037,True,Raja Aur Runk,"[sanjeev kumar, ajit, mukri, mohan choti]","[kolli pratyagatma, r.v. sastri, mukhram sharma]","[drama, family, musical]",1968,N/A,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [532]:
movie_book_dict['https://en.wikipedia.org/wiki/Race_to_Witch_Mountain']

{'movie_title': 'Race to Witch Mountain',
 'book': 'Escape to Witch Mountain',
 'imdb_no': 'tt1075417',
 'usable': True,
 'imdb_title': 'Race to Witch Mountain',
 'imdb_actors': ['dwayne johnson',
  'annasophia robb',
  'alexander ludwig',
  'carla gugino'],
 'imdb_writers': ['matt lopez', 'mark bomback', 'matt lopez', 'alexander key'],
 'imdb_genres': ['action',
  'adventure',
  'family',
  'fantasy',
  'sci-fi',
  'thriller'],
 'imdb_year': '2009',
 'imdb_rated': 'PG',
 'imdb_released': '13 Mar 2009',
 'imdb_runtime': '98 min',
 'imdb_imdbrating': '5.7',
 'imdb_imdbvotes': '51,151',
 'metadata_updated': True}

In [538]:
book_film_dict['A Clockwork Orange (1962)']

{'author': 'Anthony Burgess',
 'book_title': 'A Clockwork Orange (novel)',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/A_Clockwork_Orange_(novel)',
 'count': 2,
 'adaptations': ['Vinyl (1965 film)', 'A Clockwork Orange (film)'],
 'isbn': '0434098000',
 'oclc': '',
 'valid_identifier': True,
 'valid_identifer': True}

In [539]:
movie_book_dict['https://en.wikipedia.org/wiki/A_Clockwork_Orange_(film)']

{'movie_title': 'A Clockwork Orange (film)',
 'book': 'A Clockwork Orange (novel)',
 'imdb_no': 'tt0066921',
 'usable': True,
 'imdb_title': 'A Clockwork Orange',
 'imdb_actors': ['malcolm mcdowell',
  'patrick magee',
  'michael bates',
  'warren clarke'],
 'imdb_writers': ['stanley kubrick', 'anthony burgess'],
 'imdb_genres': ['crime', 'drama', 'sci-fi'],
 'imdb_year': '1971',
 'imdb_rated': 'R',
 'imdb_released': '02 Feb 1972',
 'imdb_runtime': '136 min',
 'imdb_imdbrating': '8.3',
 'imdb_imdbvotes': '691,997',
 'metadata_updated': True}